In [1]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import calendar
start_time = pd.datetime.now()

In [2]:
df = pd.read_excel('card transactions.xlsx')

## I. Data Preprocessing

In [3]:
df.shape

(96753, 10)

In [4]:
df.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
0,1,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0
3,4,5142148452,2010-01-01,5509006296254,FEDEX SHP 12/28/09 AB#,TN,38118.0,P,3.62,0
4,5,5142190439,2010-01-01,5509006296254,FEDEX SHP 12/23/09 AB#,TN,38118.0,P,3.62,0


In [5]:
df.dtypes

Recnum                        int64
Cardnum                       int64
Date                 datetime64[ns]
Merchnum                     object
Merch description            object
Merch state                  object
Merch zip                   float64
Transtype                    object
Amount                      float64
Fraud                         int64
dtype: object

#### 1. Transaction type & outlier

In [6]:
df.Transtype.value_counts().to_frame()

,Transtype
P,96398
A,181
D,173
Y,1


In [7]:
df = df[df.Transtype == 'P']

In [8]:
df.sort_values('Amount', ascending = False).head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
52714,52715,5142189135,2010-07-13,NaN,INTERMEXICO,NaN,NaN,P,3102045.53,0
47339,47340,5142275225,2010-06-22,NaN,INTERSOL,NaN,NaN,P,47900.00,0
59516,59517,5142138135,2010-08-03,900009045549,KRACKELER SCIENTIFIC,NY,12202.0,P,30372.46,1
80886,80887,5142152857,2010-10-11,NaN,CANYON ELECTRIC,NV,89102.0,P,28392.84,1
89673,89674,5142183210,2010-11-29,NaN,CRUCIAL TECH./MICRON,ID,83642.0,P,27218.00,0


In [9]:
df = df[df.Amount < 3102045.53]

In [10]:
df.shape

(96397, 10)

In [14]:
## display null values and zeros; note there are different dtypes
## Don't worry about the warning, it's just a disagreement b/w numpy and native python

null_zero = pd.DataFrame(dict(zip(['# of null', '# of zeros'], [df.isna().sum(), ((df == 0) | (df == '0')).sum()])))
null_zero['Total'] = null_zero['# of null']+null_zero['# of zeros']
null_zero

/Users/aslanshi/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,# of null,# of zeros,Total
Recnum,0,0,0
Cardnum,0,0,0
Date,0,0,0
Merchnum,3198,53,3251
Merch description,0,0,0
Merch state,1020,0,1020
Merch zip,4300,0,4300
Transtype,0,0,0
Amount,0,0,0
Fraud,0,95338,95338


## II. Filling in Missing Values

Before building other varibles, I need to deal with missing values.

### 3.1 Merchnum

This is hard for us to fill in since we aren't able to determine how they were designed. So I treat them as frivolous and replace them with record number.

In [15]:
def merchnum_fill(x):
    
    mernum = x[0]
    mer = x[1]
    state = x[2]
    zipcode = x[3]
    
    if pd.isnull(mernum) or mernum == '0':
        
        ## first try desp & zip
        temp = df[(df['Merch description']==mer)&(df['Merch zip']==zipcode)]
        mernums = temp['Merchnum'].value_counts()
        try:
            mf_mernum = mernums.idxmax()
        except:
            mf_mernum = mernum
            
        ## then try desp & state
        if pd.isnull(mf_mernum) or mf_mernum == '0':
            
            temp = df[(df['Merch description']==mer)&(df['Merch state']==state)]
            mernums = temp['Merchnum'].value_counts()
            try:
                mf_mernum = mernums.idxmax()
            except:
                mf_mernum = mernum

            ## finally, try desp only
            if pd.isnull(mf_mernum) or mf_mernum == '0':

                temp = df[df['Merch description']==mer]
                mernums = temp['Merchnum'].value_counts()
                try:
                    mf_mernum = mernums.idxmax()
                except:
                    mf_mernum = mernum
                    
        return mf_mernum
    
    else:
        return mernum

In [16]:
df['Merchnum'] = df[['Merchnum', 'Merch description', 'Merch state', 'Merch zip']]\
                                    .apply(merchnum_fill, axis = 1)  

In [17]:
len(df[df.Merchnum.isna()]), len(df[df.Merchnum == '0'])

(2038, 56)

In [21]:
df.loc[(df.Merchnum.isna()) | (df.Merchnum == '0'), 'Merchnum'] \
                = (df.loc[(df.Merchnum.isna()) | (df.Merchnum == '0'), 'Recnum'].values)*-1

In [24]:
len(df[df.Merchnum.isna()]), len(df[df.Merchnum == '0'])

(0, 0)

In [25]:
df.Merchnum = df.Merchnum.astype(str)

### 3.2 State & Zip

After some exploration, I found that for some records that miss state, or zip, or both, the associated merch desciption have such information in the dataset somewhere. So I would try to replace them with the most frequent. If there is no info, I will leave them as NaN.

**Important:** In real world practice, I should take validation set out and work on train/test set only.

In [26]:
df['Merch zip'] = df['Merch zip'].astype('Int64')

In [27]:
new = df.copy()

In [28]:
## value_counts() will not count NaN
## first try desp & mernum
## don't need use zipcode/state as conditions for grouping, b/c I assume merchnum has pretty low granularity

def inital_fill(x):
    
    mernum = x[0]
    mer = x[1]
    state = x[2]
    zipcode = x[3]
    
    na = pd.isnull(state) + pd.isnull(zipcode)
    
    if na == 2:
        temp = df[(df['Merchnum'] == mernum)&(df['Merch description'] == mer)]
        states = temp['Merch state'].value_counts()
        zips = temp['Merch zip'].value_counts()
        try:
            mf_state = states.idxmax()
        except:
            mf_state = state
        try:
            mf_zip = zips.idxmax()
        except:
            mf_zip = zipcode
        return [mf_state, mf_zip]
    
    elif pd.isnull(state):
        temp = df[(df['Merchnum'] == mernum)&(df['Merch description'] == mer)]
        states = temp['Merch state'].value_counts()
        try:
            mf_state = states.idxmax()
        except:
            mf_state = state
        return [mf_state, zipcode]

    elif pd.isnull(zipcode):
        temp = df[(df['Merchnum'] == mernum)&(df['Merch description'] == mer)]
        zips = temp['Merch zip'].value_counts()
        try:
            mf_zip = zips.idxmax()
        except:
            mf_zip = zipcode
        return [state, mf_zip]
    
    else:
        return [state, zipcode]

In [29]:
new[['Merch state', 'Merch zip']] = new[['Merchnum', 'Merch description', 'Merch state', 'Merch zip']]\
                                    .apply(lambda x: pd.Series(inital_fill(x)), axis = 1) 

In [30]:
new['Merch state'].isna().sum(), new['Merch zip'].isna().sum()

(1019, 2456)

In [31]:
## then only desp
## for those has null state but non-null zipcode, I also want to group on zipcode; 
## same for null zip but non-null state

def second_fill(x):
    
    mer = x[0]
    state = x[1]
    zipcode = x[2]
    
    na = pd.isnull(state) + pd.isnull(zipcode)
    
    if na == 2:
        temp = df[df['Merch description'] == mer]
        states = temp['Merch state'].value_counts()
        zips = temp['Merch zip'].value_counts()
        try:
            mf_state = states.idxmax()
        except:
            mf_state = state
        try:
            mf_zip = zips.idxmax()
        except:
            mf_zip = zipcode
        return [mf_state, mf_zip]
    
    elif pd.isnull(state):
        temp = df[(df['Merch description'] == mer)&(df['Merch zip'] == zipcode)]
        states = temp['Merch state'].value_counts()
        try:
            mf_state = states.idxmax()
        except:
            mf_state = state
        return [mf_state, zipcode]

    elif pd.isnull(zipcode):
        temp = df[(df['Merch description'] == mer)&(df['Merch state'] == state)]
        zips = temp['Merch zip'].value_counts()
        try:
            mf_zip = zips.idxmax()
        except:
            mf_zip = zipcode
        return [state, mf_zip]
    
    else:
        return [state, zipcode]

In [32]:
new[['Merch state', 'Merch zip']] = new[['Merch description', 'Merch state', 'Merch zip']]\
                                    .apply(lambda x: pd.Series(second_fill(x)), axis = 1)                             

In [33]:
new['Merch state'].isna().sum(), new['Merch zip'].isna().sum()

(364, 2054)

In [34]:
def third_fill(x):
    
    mer = x[0]
    state = x[1]
    zipcode = x[2]
    
    na = pd.isnull(state) + pd.isnull(zipcode)
    
    if na == 0 or na == 2:
        return [state, zipcode]
    
    elif pd.isnull(state):
        temp = df[df['Merch description'] == mer]
        states = temp['Merch state'].value_counts()
        try:
            mf_state = states.idxmax()
        except:
            mf_state = state
        return [mf_state, zipcode]

    else:
        temp = df[df['Merch description'] == mer]
        zips = temp['Merch zip'].value_counts()
        try:
            mf_zip = zips.idxmax()
        except:
            mf_zip = zipcode
        return [state, mf_zip]

In [35]:
new[['Merch state', 'Merch zip']] = new[['Merch description', 'Merch state', 'Merch zip']]\
                                    .apply(lambda x: pd.Series(third_fill(x)), axis = 1)   

In [36]:
new['Merch state'].isna().sum(), new['Merch zip'].isna().sum()

(363, 2043)

Now, for the remaining missing records (that miss in state or zip or both), we could not find information using their merchant description and merchant number in the dataset.

For those not missing both, I found a nice library called **uszipcode**, which could return zip given state, and ves versa. Note, there are 1 or 2 or 3 digit zipcodes, and are also some zipcodes that dont't seem to be valid. Also, there are states that don't seem to be valid. I will exclude them in the following function and deal with them later, **in case the library returns something invalid**. **The library uses some fuzzy match for state input sometimes**.

The documentary page of the library is here: https://pypi.org/project/uszipcode/

**Special case study:**

In [54]:
## For example, AB is short for Alberta, which is a province in Canada
## however the liabray will fuzzy match it to AL, Alabama
## this is not desirable

searcher.by_state('AB')[0]

SimpleZipcode(zipcode='35004', zipcode_type='Standard', major_city='Moody', post_office_city='Moody, AL', common_city_list=['Moody', 'Acmar'], county='St. Clair County', state='AL', lat=33.62, lng=-86.5, timezone='Central', radius_in_miles=4.0, area_code_list=['205'], population=10427, population_density=577.0, land_area_in_sqmi=18.07, water_area_in_sqmi=0.14, housing_units=4523, occupied_housing_units=4214, median_home_value=142500, median_household_income=58832, bounds_west=-86.551776, bounds_east=-86.452822, bounds_north=33.668501, bounds_south=33.562686)

---- end ----

In [37]:
## there are some states that are not valid us states...

new[(new['Merch state'].notna()) & (new['Merch zip'].isna())]['Merch state'].unique()

array(['NY', 'GA', 'MA', 'OK', 'MD', 'BC', 'KY', 'CA', 'TX', 'QC', 'MO',
       'PA', 'ON', 'VA', 'MS', 'IL', 'NC', 'WV', 'FL', 'KS', 'CO', 'MB',
       'OR', 'WA', 'CT', 'WI', 'AZ', 'DC', 'IN', 'IA', 'MI', 'NJ', 'PQ',
       'MN', 'AL', 'OH', 'RI', 'MT', 'UT', 'NS', 'AK', 'NV', 'AB', 'NM',
       'LA', 'ID', 'ND', 'NE'], dtype=object)

In [38]:
## I use webscrapping to create a list of valid US abbrv.

from bs4 import BeautifulSoup
from urllib.request import urlopen

In [39]:
html = urlopen("https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations")
soup = BeautifulSoup(html)

In [40]:
table = soup.find("table", {'class':'wikitable sortable'})

In [41]:
names = table.find_all("span", {'class': 'monospaced'})
names = names[1:-15]

In [42]:
state_list = []
for name in names:
    name = name.get_text()[-2:]
    state_list.append(name)

In [44]:
from uszipcode import SearchEngine
searcher = SearchEngine()

In [45]:
new1 = new.copy()

In [60]:
def state_zip_fill(x):
    
    state = x[0]
    zipcode = x[1]
    
    na = pd.isnull(state) + pd.isnull(zipcode)
    
    if na == 1:
        if pd.isnull(state):
            result = searcher.by_zipcode(str(int(zipcode))).state
            if result == None:
                return [state, zipcode]
            else:
                return [result, zipcode]
        else:
            if state not in state_list:
                return [state, zipcode]
            else:
                result = searcher.by_state(state)[0].zipcode
                if result == None:
                    return [state, zipcode]
                else:
                    return [state, result]
    else:
        return [state, zipcode]

In [61]:
new1[['Merch state', 'Merch zip']] = new1[['Merch state', 'Merch zip']]\
                                    .apply(lambda x: pd.Series(state_zip_fill(x)), axis = 1) 

In [62]:
new1['Merch state'].isna().sum(), new1['Merch zip'].isna().sum()

(310, 454)

In [63]:
## There are 269 records that miss both entities information at this point, and there is no valuable information
## that we could use in the dataset; one of the approach I came up with is to find the associated cardnum and 
## replace the null with the most frequent value used by that card
## I am NOT 100% sure about this but unless there is better approach I will use this.

len(new1[(new1['Merch state'].isna()) & (new1['Merch zip'].isna())])

269

In [64]:
## I did some research and found that these are likely to be province codes of Canda
## I decide to manully set these zipcodes to be the state, since it's likely that when new data come in,
## these zip codes would be left as null as well, so just make them state as well.

new1[(new1['Merch zip'].isna()) & (new1['Merch state'].notna())]['Merch state'].unique()

array(['BC', 'QC', 'ON', 'MB', 'PQ', 'NS', 'AB'], dtype=object)

In [65]:
new1[(new1['Merch zip'].notna()) & (new1['Merch state'].isna())]['Merch zip'].unique()

array([1400.0, 65132.0, 86899.0, 23080.0, 60528.0, 6.0, 8.0, 9.0, 50823.0,
       2.0, 48700.0, 1.0, 3.0, 801.0], dtype=object)

In [66]:
new2 = new1.copy()

May be we don't want to fill in using record. This is because for these weird zipcodes, it's possible that they're made up. So instead of reducing its chance getting detected by the model, we'd better keep the original values and later in our concatenation of entities, they will appear as what they are.

In [67]:
## for the state above, I will first try to find records associated with them in the whole dataset
## if there is any, I will replace with the most frequent
## note, this time the granularity is different, records are not sorted by merchant descrip but by state

def zip_fill(x):
    
    state = x[0]
    zipcode = x[1]
    
    na = pd.isnull(state) + pd.isnull(zipcode)
    
    if na == 1 and pd.isnull(zipcode):
        temp = new2[new2['Merch state'] == state]
        zips = temp['Merch zip'].value_counts()
        try:
            mf_zip = zips.idxmax()
            return [state, mf_zip]
        except:
            return [state, state]  ## note here I change it to state
    else:
        return [state, zipcode]

In [68]:
new2[['Merch state', 'Merch zip']] = new2[['Merch state', 'Merch zip']]\
                                    .apply(lambda x: pd.Series(zip_fill(x)), axis = 1) 

In [69]:
## Similarly, I will first try to find records associated with the zip codes in the whole dataset
## if there is any, I will replace with the most frequent
## else, I will manually assign the state as string zipcode

def state_fill(x):
    
    state = x[0]
    zipcode = x[1]
    
    na = pd.isnull(state) + pd.isnull(zipcode)
    
    if na == 1 and pd.isnull(state):
        temp = new2[new2['Merch zip'] == zipcode]
        states = temp['Merch state'].value_counts()
        try:
            mf_state = states.idxmax()
            return [mf_state, zipcode]
        except:
            return [str(int(zipcode)), zipcode] ## note here I use zipcode as state
    else:
        return [state, zipcode]

In [70]:
new2[['Merch state', 'Merch zip']] = new2[['Merch state', 'Merch zip']]\
                                    .apply(lambda x: pd.Series(state_fill(x)), axis = 1) 

In [71]:
## if the above approach went successful, the following 3 numbers should all = 269
## which is the final remaining number of records that
##   1. their merchant descriptioon provide no information on state and zip in the whole dataset
##   2. they miss both state and zip code

len(new2[(new2['Merch state'].isna())]), len(new2[new2['Merch zip'].isna()]),\
                            len(new2[(new2['Merch state'].isna()) & (new2['Merch zip'].isna())])

(269, 269, 269)

In [72]:
## after checking with professor, the approach i will use is to find the associated cardnum and 
## replace the null with the most frequent value used by that card

new3 = new2.copy()

In [79]:
def final_fill(x):
    
    temp = new3[new3.Cardnum == x]
    try:
        mf_state = temp['Merch state'].value_counts().idxmax()
    except:
        mf_state = np.nan
    try:
        mf_zip = temp['Merch zip'].value_counts().idxmax()
    except:
        mf_zip = np.nan
        
    return pd.Series([mf_state, mf_zip])

In [80]:
nulls_df = new3[(new3['Merch state'].isna()) & (new3['Merch zip'].isna())]

null_merchs = nulls_df['Merch description'].unique()

for merch in null_merchs:
    
    temp = new3.loc[new3['Merch description'] == merch, 'Cardnum']
    result = temp.apply(final_fill).values
    new3.loc[new3['Merch description'] == merch, ['Merch state', 'Merch zip']] = result

In [81]:
pd.DataFrame(dict(zip(['# of null', '# of zeros'], [new3.isna().sum(), ((new3 == 0) | (new3 == '0')).sum()])))

/Users/aslanshi/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,# of null,# of zeros
Recnum,0,0
Cardnum,0,0
Date,0,0
Merchnum,0,0
Merch description,0,0
Merch state,31,0
Merch zip,31,0
Transtype,0,0
Amount,0,0
Fraud,0,95338


In [87]:
## for the last 31 records, I will replace with negative record number

new3.loc[(new3['Merch state'].isna())&(new3['Merch zip'].isna()), 'Merch state'] =\
    (new3.loc[(new3['Merch state'].isna())&(new3['Merch zip'].isna()),'Recnum'].values)*-1
new3.loc[new3['Merch zip'].isna(), 'Merch zip'] =\
    (new3.loc[new3['Merch zip'].isna(),'Recnum'].values)*-1

In [89]:
## Finally.....

pd.DataFrame(dict(zip(['# of null', '# of zeros'], [new3.isna().sum(), ((new3 == 0) | (new3 == '0')).sum()])))

,# of null,# of zeros
Recnum,0,0
Cardnum,0,0
Date,0,0
Merchnum,0,0
Merch description,0,0
Merch state,0,0
Merch zip,0,0
Transtype,0,0
Amount,0,0
Fraud,0,95338


In [90]:
new3.shape

(96397, 10)

In [91]:
new3.to_csv('cleaned_data_v2.csv')

In [51]:
print('Duration: ', pd.datetime.now() - start_time)

Duration:  0:02:08.503877


In [92]:
new3[new3['Merch zip']=='AB']

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud
44963,44964,5142117641,2010-06-15,-44964,AGRICULTURE FOOD & RUR,AB,AB,P,37.00,0
50776,50777,5142291592,2010-07-06,-50777,SEQUITER SOFTWARE INC,AB,AB,P,752.83,0
80132,80133,5142251947,2010-10-06,-80133,TROUT UNLIMITED CANADA,AB,AB,P,225.00,0
83155,83156,5142254784,2010-10-25,-83156,ENVIRONMENTAL SERVICES AS,AB,AB,P,527.00,0
89916,89917,5142111372,2010-11-30,-89917,COMMUNICATION SYSTEMS,AB,AB,P,48.58,0
